In [1]:
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import pymongo

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\rohti\.wdm\drivers\chromedriver\win32\89.0.4389.23]


# Current Listing

In [5]:
current_listing_urls = ["https://www.redfin.com/city/30844/TX/Frisco",
       "https://www.redfin.com/city/30794/TX/Dallas",
       "https://www.redfin.com/city/30827/TX/Fort-Worth",
       "https://www.redfin.com/city/30868/TX/Plano",
       "https://www.redfin.com/city/1067/TX/Arlington",
       "https://www.redfin.com/city/9410/TX/Irving",
       "https://www.redfin.com/city/30852/TX/Southlake",
       "https://www.redfin.com/city/11666/TX/McKinney",
       "https://www.redfin.com/city/4139/TX/Colleyville",
       "https://www.redfin.com/city/30867/TX/Coppell",
       "https://www.redfin.com/city/30837/TX/Flower-Mound"]

In [11]:
price_list = []
zipcode_list = []
address_list = []
beds_list = []
baths_list = []
sq_ft_list = []
    
for url in current_listing_urls:
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Price of listed properties
    price = soup.find_all('span', class_="homecardV2Price")
    
    # Numebr of Beds, Baths and Sq. Ft. of listed properties
    stats = soup.find_all('div', class_="stats")
    
    # Address of listed properties
    address = soup.find_all('div', class_="link-and-anchor")
    
    # Convering Price value into numeric - removing "$" and ","
    for p in price:
        price_list.append(float(p.text.replace("$","").replace(",","").replace("+","").strip()))
    
    # Convering Number of Beds, Number of Baths and Area Sq. Ft. values into numeric - removing text: "Beds", "Baths", "Sq. Ft."

    for each in stats:
        val = each.text.strip()
        if 'Bed' in val:
            beds_list.append(float(val.replace("Beds","").replace("Bed","").replace('—','2').strip()))
        elif 'Bath' in val:  
            baths_list.append(float(val.replace("Baths","").replace("Bath","").replace('—','2').strip()))
        elif 'Sq. Ft.' in val:
            sq_ft_list.append(float(val.replace("Sq. Ft.","").replace(",","").replace('—','1000').strip()))
    
    for z in address:
        zipcode_list.append(z.text[-5:].strip())
        address_list.append(z.text.strip())
    
    print("completed url", url, "with records", len(price_list))

    

    
print("Number of records in price list", len(price_list))
print("Number of records in beds list", len(beds_list))
print("Number of records in baths list", len(baths_list))
print("Number of records in sq. ft. list", len(sq_ft_list))
print("Number of records in zipcode list", len(zipcode_list))
print("Number of records in zipcode list", len(address_list))

completed url https://www.redfin.com/city/30844/TX/Frisco with records 41
completed url https://www.redfin.com/city/30794/TX/Dallas with records 82
completed url https://www.redfin.com/city/30827/TX/Fort-Worth with records 123
completed url https://www.redfin.com/city/30868/TX/Plano with records 164
completed url https://www.redfin.com/city/1067/TX/Arlington with records 205
completed url https://www.redfin.com/city/9410/TX/Irving with records 246
completed url https://www.redfin.com/city/30852/TX/Southlake with records 287
completed url https://www.redfin.com/city/11666/TX/McKinney with records 328
completed url https://www.redfin.com/city/4139/TX/Colleyville with records 369
completed url https://www.redfin.com/city/30867/TX/Coppell with records 397
completed url https://www.redfin.com/city/30837/TX/Flower-Mound with records 438
Number of records in price list 438
Number of records in beds list 438
Number of records in baths list 438
Number of records in sq. ft. list 438
Number of re

In [62]:
current_listing = []

for i in range(len(zipcode_list)):
    temp_dict = {"ZipCode":zipcode_list[i], "NumberOfBeds": beds_list[i], "NumberOfBaths": baths_list[i], 
           "SqFt": sq_ft_list[i], "Price":price_list[i], "Address":address_list[i], "City":city_list[i]}
    current_listing.append(temp_dict)
    
print("Records to insert in DB:")
print(current_listing[:5])

Records to insert in DB:
[{'ZipCode': '75035', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.5, 'SqFt': 2527.0, 'Price': 420000.0, 'Address': '12174 Shepherds Hill Ln, Frisco, TX 75035', 'City': 'Frisco'}, {'ZipCode': '75034', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 2102.0, 'Price': 399000.0, 'Address': '4617 Chandler Dr, Frisco, TX 75034', 'City': 'Frisco'}, {'ZipCode': '75035', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1785.0, 'Price': 375000.0, 'Address': '7599 Petersburgh Pl, Frisco, TX 75035', 'City': 'Frisco'}, {'ZipCode': '75034', 'NumberOfBeds': 4.0, 'NumberOfBaths': 2.0, 'SqFt': 2319.0, 'Price': 399900.0, 'Address': '3493 Mayflower Dr, Frisco, TX 75034', 'City': 'Frisco'}, {'ZipCode': '75034', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.5, 'SqFt': 4344.0, 'Price': 950000.0, 'Address': '5145 Shoreline Dr, Frisco, TX 75034', 'City': 'Frisco'}]


In [63]:
current_listing_df = pd.DataFrame(current_listing)
current_listing_df.to_csv('current_listing_05082021_v2.csv')

In [64]:
import json
with open("current_listing_05082021_v2.json", "w") as outfile: 
    json.dump(current_listing, outfile)

# Sold Listing

In [6]:
sold_listing_urls = [i+"/filter/include=sold-6mo" for i in current_listing_urls]
sold_listing_urls

['https://www.redfin.com/city/30844/TX/Frisco/filter/include=sold-6mo',
 'https://www.redfin.com/city/30794/TX/Dallas/filter/include=sold-6mo',
 'https://www.redfin.com/city/30827/TX/Fort-Worth/filter/include=sold-6mo',
 'https://www.redfin.com/city/30868/TX/Plano/filter/include=sold-6mo',
 'https://www.redfin.com/city/1067/TX/Arlington/filter/include=sold-6mo',
 'https://www.redfin.com/city/9410/TX/Irving/filter/include=sold-6mo',
 'https://www.redfin.com/city/30852/TX/Southlake/filter/include=sold-6mo',
 'https://www.redfin.com/city/11666/TX/McKinney/filter/include=sold-6mo',
 'https://www.redfin.com/city/4139/TX/Colleyville/filter/include=sold-6mo',
 'https://www.redfin.com/city/30867/TX/Coppell/filter/include=sold-6mo',
 'https://www.redfin.com/city/30837/TX/Flower-Mound/filter/include=sold-6mo']

In [12]:
price_list_sold = []
zipcode_list_sold = []
address_list_sold = []
beds_list_sold = []
baths_list_sold = []
sq_ft_list_sold = []
    
for url in sold_listing_urls:
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Price of listed properties
    price_sold = soup.find_all('span', class_="homecardV2Price")
    
    # Numebr of Beds, Baths and Sq. Ft. of listed properties
    stats_sold = soup.find_all('div', class_="stats")
    
    # Address of listed properties
    address_sold = soup.find_all('div', class_="link-and-anchor")
    
    # Convering Price value into numeric - removing "$" and ","
    for p in price_sold:
        price_list_sold.append(float(p.text.replace("$","").replace(",","").replace("+","").strip()))
    
    # Convering Number of Beds, Number of Baths and Area Sq. Ft. values into numeric - removing text: "Beds", "Baths", "Sq. Ft."

    for each in stats_sold:
        val = each.text.strip()
        if 'Bed' in val:
            beds_list_sold.append(float(val.replace("Beds","").replace("Bed","").replace('—','2').strip()))
        elif 'Bath' in val:  
            baths_list_sold.append(float(val.replace("Baths","").replace("Bath","").replace('—','2').strip()))
        elif 'Sq. Ft.' in val:
            sq_ft_list_sold.append(float(val.replace("Sq. Ft.","").replace(",","").replace('—','1000').strip()))
    
    for z in address_sold:
        zipcode_list_sold.append(z.text[-5:].strip())
        address_list_sold.append(z.text.strip())
    
    print("completed url", url, "with records", len(price_list_sold))

    

    
print("Number of records in price list", len(price_list_sold))
print("Number of records in beds list", len(beds_list_sold))
print("Number of records in baths list", len(baths_list_sold))
print("Number of records in sq. ft. list", len(sq_ft_list_sold))
print("Number of records in zipcode list", len(zipcode_list_sold))
print("Number of records in zipcode list", len(address_list_sold))

completed url https://www.redfin.com/city/30844/TX/Frisco/filter/include=sold-6mo with records 41
completed url https://www.redfin.com/city/30794/TX/Dallas/filter/include=sold-6mo with records 82
completed url https://www.redfin.com/city/30827/TX/Fort-Worth/filter/include=sold-6mo with records 123
completed url https://www.redfin.com/city/30868/TX/Plano/filter/include=sold-6mo with records 164
completed url https://www.redfin.com/city/1067/TX/Arlington/filter/include=sold-6mo with records 205
completed url https://www.redfin.com/city/9410/TX/Irving/filter/include=sold-6mo with records 246
completed url https://www.redfin.com/city/30852/TX/Southlake/filter/include=sold-6mo with records 287
completed url https://www.redfin.com/city/11666/TX/McKinney/filter/include=sold-6mo with records 328
completed url https://www.redfin.com/city/4139/TX/Colleyville/filter/include=sold-6mo with records 369
completed url https://www.redfin.com/city/30867/TX/Coppell/filter/include=sold-6mo with records 41

In [13]:
browser.quit()

In [65]:
sold_listing = []

for i in range(len(zipcode_list_sold)):
    temp_dict = {"ZipCode":zipcode_list_sold[i], "NumberOfBeds": beds_list_sold[i], "NumberOfBaths": baths_list_sold[i], 
           "SqFt": sq_ft_list_sold[i], "Price":price_list_sold[i], "Address":address_list_sold[i], "City":city_list_sold[i]}
    sold_listing.append(temp_dict)
    
print("Records to insert in DB:")
print(sold_listing[:5])

Records to insert in DB:
[{'ZipCode': '75036', 'NumberOfBeds': 5.0, 'NumberOfBaths': 5.0, 'SqFt': 4482.0, 'Price': 780000.0, 'Address': '5395 Statesman Ln, Frisco, TX 75036', 'City': 'Frisco'}, {'ZipCode': '75036', 'NumberOfBeds': 4.0, 'NumberOfBaths': 2.0, 'SqFt': 2018.0, 'Price': 325000.0, 'Address': '11509 Parade Dr, Frisco, TX 75036', 'City': 'Frisco'}, {'ZipCode': '75033', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.0, 'SqFt': 3196.0, 'Price': 400000.0, 'Address': '12759 Blue Ridge Dr, Frisco, TX 75033', 'City': 'Frisco'}, {'ZipCode': '75035', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1660.0, 'Price': 274900.0, 'Address': '13003 Michelle Dr, Frisco, TX 75035', 'City': 'Frisco'}, {'ZipCode': '75033', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.5, 'SqFt': 3576.0, 'Price': 489900.0, 'Address': '1186 Shortgrass Ln, Frisco, TX 75033', 'City': 'Frisco'}]


In [66]:
sold_listing_df = pd.DataFrame(sold_listing)
sold_listing_df.to_csv('sold_listing_05082021_v2.csv')

In [68]:
with open("sold_listing_05082021_v2.json", "w") as outfile: 
    json.dump(sold_listing, outfile)

# Getting Current Listing Coordinates

In [23]:
import requests
import pandas as pd
import numpy as np

In [30]:
city_list = [i.split(",")[1].strip() for i in address_list]
len(city_list)

438

In [51]:
url = "https://api.geocod.io/v1.6/geocode"

current_listing_coord = []
coordinates_new = []

for i in range(len(city_list)):
    try:
        address_dummy =address_list[i].replace(" ", "+")
        city_dummy = city_list[i]
    
        response = requests.get(f"{url}?q={address_dummy}+%2c+{city_dummy}&api_key=00822cf80e63d6daa666f6c9c90f0a029d3f8d3").json()
        #print(response)
        
        coordinates_new = [response["results"][0]["location"]["lat"],response["results"][0]["location"]["lng"]]
        
        temp_dict = {"coordinates":coordinates_new, "ZipCode":zipcode_list[i], "NumberOfBeds": str(beds_list[i]), "NumberOfBaths": str(baths_list[i]), 
           "SqFt": str(sq_ft_list[i]), "price":str(price_list[i]), "Address":str(address_list[i])}
        
        current_listing_coord.append(temp_dict)
        
       
        
        print("coords added")
    except:
        print(f"City not found. Skipping...")

coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
City not found. Skipping...
coords added
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coor

In [52]:
len(current_listing_coord)

424

In [53]:
current_listing_coord[:5]

[{'coordinates': [33.142374, -96.772484],
  'ZipCode': '75035',
  'NumberOfBeds': '3.0',
  'NumberOfBaths': '2.5',
  'SqFt': '2527.0',
  'price': '420000.0',
  'Address': '12174 Shepherds Hill Ln, Frisco, TX 75035'},
 {'coordinates': [33.117505, -96.856639],
  'ZipCode': '75034',
  'NumberOfBeds': '3.0',
  'NumberOfBaths': '2.0',
  'SqFt': '2102.0',
  'price': '399000.0',
  'Address': '4617 Chandler Dr, Frisco, TX 75034'},
 {'coordinates': [33.140917, -96.80087],
  'ZipCode': '75035',
  'NumberOfBeds': '3.0',
  'NumberOfBaths': '2.0',
  'SqFt': '1785.0',
  'price': '375000.0',
  'Address': '7599 Petersburgh Pl, Frisco, TX 75035'},
 {'coordinates': [33.108031, -96.857877],
  'ZipCode': '75034',
  'NumberOfBeds': '4.0',
  'NumberOfBaths': '2.0',
  'SqFt': '2319.0',
  'price': '399900.0',
  'Address': '3493 Mayflower Dr, Frisco, TX 75034'},
 {'coordinates': [33.12057, -96.841183],
  'ZipCode': '75034',
  'NumberOfBeds': '4.0',
  'NumberOfBaths': '3.5',
  'SqFt': '4344.0',
  'price': '9500

In [55]:
with open("current_listings_coord_05082021.json", "w") as outfile: 
    json.dump(current_listing_coord, outfile)

# Getting Sold Listing Coordinates

In [54]:
city_list_sold = [i.split(",")[1].strip() for i in address_list_sold]
len(city_list_sold)

451

In [56]:
url = "https://api.geocod.io/v1.6/geocode"

sold_listing_coord = []
coordinates_new_sold = []

for i in range(len(city_list_sold)):
    try:
        address_dummy =address_list_sold[i].replace(" ", "+")
        city_dummy = city_list_sold[i]
    
        response = requests.get(f"{url}?q={address_dummy}+%2c+{city_dummy}&api_key=00822cf80e63d6daa666f6c9c90f0a029d3f8d3").json()
        #print(response)
        
        coordinates_new_sold = [response["results"][0]["location"]["lat"],response["results"][0]["location"]["lng"]]
        
        temp_dict = {"coordinates":coordinates_new_sold, "ZipCode":zipcode_list_sold[i], "NumberOfBeds": str(beds_list_sold[i]), "NumberOfBaths": str(baths_list_sold[i]), 
           "SqFt": str(sq_ft_list_sold[i]), "price":str(price_list_sold[i]), "Address":str(address_list_sold[i])}
        
        sold_listing_coord.append(temp_dict)
        
       
        
        print("coords added")
    except:
        print(f"City not found. Skipping...")

coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
City not found. Skipping...
coords added
coords added
coords added
coords added
Ci

In [57]:
len(sold_listing_coord)

435

In [58]:
sold_listing_coord[:5]

[{'coordinates': [33.123621, -96.886309],
  'ZipCode': '75036',
  'NumberOfBeds': '5.0',
  'NumberOfBaths': '5.0',
  'SqFt': '4482.0',
  'price': '780000.0',
  'Address': '5395 Statesman Ln, Frisco, TX 75036'},
 {'coordinates': [33.139863, -96.93226],
  'ZipCode': '75036',
  'NumberOfBeds': '4.0',
  'NumberOfBaths': '2.0',
  'SqFt': '2018.0',
  'price': '325000.0',
  'Address': '11509 Parade Dr, Frisco, TX 75036'},
 {'coordinates': [33.184743, -96.874211],
  'ZipCode': '75033',
  'NumberOfBeds': '4.0',
  'NumberOfBaths': '3.0',
  'SqFt': '3196.0',
  'price': '400000.0',
  'Address': '12759 Blue Ridge Dr, Frisco, TX 75033'},
 {'coordinates': [33.184745, -96.785837],
  'ZipCode': '75035',
  'NumberOfBeds': '3.0',
  'NumberOfBaths': '2.0',
  'SqFt': '1660.0',
  'price': '274900.0',
  'Address': '13003 Michelle Dr, Frisco, TX 75035'},
 {'coordinates': [32.814018, -96.948895],
  'ZipCode': '75033',
  'NumberOfBeds': '4.0',
  'NumberOfBaths': '3.5',
  'SqFt': '3576.0',
  'price': '489900.0',

In [59]:
with open("sold_listing_coord_05082021.json", "w") as outfile: 
    json.dump(sold_listing_coord, outfile)